In [1]:
import numpy as np
import json, os
from nltk.corpus import wordnet as wn
from math import log
import glob
import tqdm
import json
import spacy
import matplotlib.pyplot as plt

In [2]:
def _getTextFile(langual):
    file_list = glob.glob(f'../data/stopwords/stopwords/*_{langual}.txt')
    files = ",".join(file_list)
    return files

def cleanText(english_txt):
    try:
        word_tokens = english_txt.split()
        filtered_word = [w for w in word_tokens if w not in stop_words and not w.isdigit()]
        filtered_word = [w + " " for w in filtered_word]
        return "".join(filtered_word)
    except:
        return np.nan

def detectLang(txt):
    try:
        return detect(txt)
    except:
        return np.nan

def cleanNonEnglish(txt):
    txt = re.sub(r'\W+', ' ', txt)
    txt = txt.lower()
    txt = txt.replace("[^a-zA-Z]", " ")
    word_tokens = txt.split()
    filtered_word = [w for w in word_tokens if all(ord(c) < 128 for c in w)]
    filtered_word = [w + " " for w in filtered_word]
    return "".join(filtered_word)

def countMinMaxAver(lines):
    min_len = 10000
    aver_len = 0
    max_len = 0
    for temp in lines:
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)
    aver_len = 1.0 * aver_len / len(lines)
    print('min_len : ' + str(min_len))
    print('max_len : ' + str(max_len))
    print('average_len : ' + str(aver_len))
    
def titleProcessing(title):
    #title = re.findall(r"title=\"(.*)\">", title)
    #title = title[0].replace(" ", "_")
    return re.findall(r"title=\"(.*)\">", title)

def contentProcessing(content, title_list_i):
    if content[0].strip() == title_list_i[0]:
        return title_list_i, content[1].strip()

In [3]:
stop_words=set()
for file in _getTextFile("en").split(","):
    for word in open(file):
        stop_words.add(word.strip())

In [4]:
tagme_json = np.load("../data/wordnet_sememe_entity_sence.npy", allow_pickle=True).tolist()
title_content_lemmatization = np.load("../data/wordnet_lemmatization_sence.npy", allow_pickle=True).tolist()

In [5]:
tagme_json[87566]

'87566\t[{"spot": "fruit", "start": 0, "link_probability": 0.10305586457252502, "rho": 0.23641960322856903, "end": 5, "id": 10843, "title": "Fruit"}, {"spot": "red", "start": 11, "link_probability": 0.012815726920962334, "rho": 0.17817823588848114, "end": 14, "id": 25825, "title": "Red"}, {"spot": "yellow", "start": 18, "link_probability": 0.015596273355185986, "rho": 0.2011156529188156, "end": 24, "id": 34368, "title": "Yellow"}, {"spot": "green", "start": 28, "link_probability": 0.02863074094057083, "rho": 0.19163185358047485, "end": 33, "id": 12460, "title": "Green"}, {"spot": "skin", "start": 34, "link_probability": 0.05207127332687378, "rho": 0.20922449231147766, "end": 38, "id": 27978, "title": "Skin"}, {"spot": "sweet", "start": 43, "link_probability": 0.021137243136763573, "rho": 0.16798323392868042, "end": 48, "id": 1781797, "title": "Sweetness"}, {"spot": "tart", "start": 52, "link_probability": 0.11760063469409943, "rho": 0.28761711716651917, "end": 56, "id": 21282070, "titl

In [6]:
len(tagme_json)

227733

In [7]:
len(title_content_lemmatization)

227733

In [8]:
title_list = list(title_content_lemmatization.keys())

In [9]:
stop_words.add("")
stop_words.add("'s")
stop_words.add("States")
stop_words.add("United")

In [10]:
tagme_sememe_dict_l = {}
for items in tqdm.tqdm(tagme_json):
    if not items:
        continue 
    i, entityList = items.split("\t")
    if entityList == "null" or len(entityList) == 0:
        continue
    entityList = json.loads(entityList)
    entities = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    #entities = [d['title'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    title = title_list[int(i)]
    if title not in tagme_sememe_dict_l.keys():
        tagme_sememe_dict_l[title] = []
    entities = " ".join(entities).strip().split(" ")
    tagme_sememe_dict_l[title] = entities

100%|████████████████████████████████| 227733/227733 [00:02<00:00, 80981.72it/s]


In [11]:
tagme_sememe_dict = {k:v for k,v in tagme_sememe_dict_l.items() if len(v)!=0}

In [12]:
len(tagme_sememe_dict)

227733

In [13]:
for k, v in tagme_sememe_dict.items():
    def_list = []
    for v_i in v:
        if v_i not in stop_words:
            def_list.append(v_i)
    tagme_sememe_dict[k]=def_list

In [14]:
word_freq = {}
word_set = set()
for doc_words in tagme_sememe_dict.values():
    for word in doc_words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

print("content lexion: ", vocab_size)

content lexion:  29629


In [15]:
word_freq_sorted = sorted(word_freq.items(), key = lambda kv:(kv[1], kv[0]))

In [16]:
sememe_raw = [word for (word, _) in word_freq_sorted[-2000:]]

In [17]:
title_sememe_raw = {}
for k, v in tqdm.tqdm(tagme_sememe_dict.items()):
    sememe = []
    for w in v:
        if w in sememe_raw:
        #if w in dict_sememes:
            sememe.append(w)
    if len(sememe) == 0:
        continue
    title_sememe_raw[k] = sememe

print("take max [:3000] in content lexion for sememe (raw)")
countMinMaxAver(title_sememe_raw)

100%|████████████████████████████████| 227733/227733 [00:14<00:00, 16036.53it/s]

take max [:3000] in content lexion for sememe (raw)
min_len : 5
max_len : 75
average_len : 14.520558344557365


In [18]:
sememe_freq = {}
sememe_set = set()
for doc_sememes in title_sememe_raw.values():
    for sememe in doc_sememes:
        sememe_set.add(sememe)
        if sememe in sememe_freq:
            sememe_freq[sememe] += 1
        else:
            sememe_freq[sememe] = 1

sememe_lexion = list(sememe_set)
sememe_lexion_size = len(sememe_lexion)

sememe_id_map = {}
for i in range(sememe_lexion_size):
    sememe_id_map[sememe_lexion[i]] = i

print("sememe lexion size: ", sememe_lexion_size)

sememe lexion size:  2000


In [19]:
doc_word_freq = {}
for doc_id,(_, doc_words) in enumerate(title_sememe_raw.items()):
    for word in doc_words:
        word_id = sememe_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1
            
word_doc_list = {}
for i,(_, doc_words) in enumerate(title_sememe_raw.items()):
    appeared = set()
    for word in doc_words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)
    
tfidf_word2doc_all = {}
for i,(title, doc_words) in enumerate(title_sememe_raw.items()):
    doc_word_set = set()
    for word in doc_words:
        if word in doc_word_set or word == title:
            continue
        j = sememe_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        idf = log(1.0 * len(title_sememe_raw) / word_doc_freq[sememe_lexion[j]])
        tfidf_word2doc_all[key] = freq * idf
        doc_word_set.add(word)

In [28]:
tfidf_word2doc_all

{'0,1722': 6.180543301429248,
 '0,1115': 6.405167237913892,
 '1,425': 5.030982945610513,
 '1,577': 6.6517310634362,
 '1,624': 5.734492913206005,
 '2,1572': 7.090934311176214,
 '2,1165': 5.704639950056324,
 '3,506': 7.026855454491693,
 '4,718': 6.344106981152645,
 '4,1807': 7.492729690395416,
 '4,529': 6.522950273570276,
 '5,605': 6.478136257780482,
 '6,605': 6.478136257780482,
 '6,1587': 5.132497902301721,
 '6,1179': 6.454513112017047,
 '6,536': 5.409175737162488,
 '6,443': 7.460294414642263,
 '6,1487': 7.713129755763876,
 '6,1970': 6.614517467095766,
 '7,515': 14.382809683079698,
 '7,398': 15.481818639741903,
 '8,486': 5.083905347064856,
 '8,210': 6.082010777268451,
 '8,280': 7.672855856625936,
 '8,808': 15.319567550117165,
 '9,1897': 12.612991420813724,
 '10,1787': 7.321457969788047,
 '10,144': 6.535529055777135,
 '10,809': 4.97096493588426,
 '10,210': 6.082010777268451,
 '11,869': 5.535890470807913,
 '11,334': 5.420179974753502,
 '11,425': 5.030982945610513,
 '12,273': 4.22705416872

In [29]:
title_list = [word for word in title_sememe_raw.keys()]
tfidf_filter = {}
for title, tfidf in tfidf_word2doc_all.items():
    if tfidf > 0:
        w, d = title.split(",")
        if title_list[int(w)] not in tfidf_filter.keys():
            tfidf_filter[title_list[int(w)]] = []
        tfidf_filter[title_list[int(w)]].append(sememe_lexion[int(d)])
print("all lexion size:", len(tfidf_filter))

all lexion size: 163340


In [21]:
# this part for add pmi weight to building graph
title_list = [word for word in title_sememe_raw.keys()]
tfidf_filter = {}
for title, tfidf in tfidf_word2doc_all.items():
    if tfidf > 0:
        w, d = title.split(",")
        if title_list[int(w)] not in tfidf_filter.keys():
            tfidf_filter[title_list[int(w)]] = []
        pmi_weight = sememe_lexion[int(d)]+":"+str(tfidf)
        tfidf_filter[title_list[int(w)]].append(pmi_weight)
print("all lexion size:", len(tfidf_filter))

all lexion size: 163340


In [30]:
c

In [31]:
print("all lexion size:", len(tfidf_filter))

all lexion size: 6


In [36]:
np.save("wordnet_sememe_163340", tfidf_filter)

In [32]:
sememe_network_pmi = {}
for k,v in tfidf_filter.items():
    for vi in v:
        vi,pmi=vi.split(":")
        if vi in sememe_network[k]:
            if k not in sememe_network_pmi.keys():
                sememe_network_pmi[k] = []
            sememe_network_pmi[k].append(vi+":"+pmi)

ValueError: not enough values to unpack (expected 2, got 1)

In [44]:
len(sememe_network_pmi)

137601

In [39]:
np.save("wordnet_sememe_163340_218", sememe_network_pmi)

In [22]:
len(title_list)

163340

In [26]:
title_id_map = {w:i for i,w in enumerate(title_list)}
max_value = 4
sememe_network = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            sememe_freq_max_value = {}
            for sememe in sememes:
                tfidf_word2doc_all_key = str(title_id_map[word]) +","+ str(sememe_id_map[sememe])
                sememe_freq_max_value[sememe] = tfidf_word2doc_all[tfidf_word2doc_all_key]
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network[word] = sememe_lower
        else:
            sememe_network[word] = sememes
    else:
        continue

KeyError: 'print:6.478136257780482'

In [24]:
title_list = list(title_content_lemmatization.keys())

In [27]:
tagme_sememe_dict = {}
for items in tqdm.tqdm(tagme_json):
    if not items:
        continue 
    i, entityList = items.split("\t")
    if entityList == "null" or len(entityList) == 0:
        continue
    entityList = json.loads(entityList)
    #entities = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    entities = [d['title']+"#"+str(d['link_probability']) for d in entityList if 'title' in d and float(d['rho']) > 0.01]
    #print(i)
    title = title_list[int(i)]
    if title not in tagme_sememe_dict.keys():
        tagme_sememe_dict[title] = []
    tagme_sememe_dict[title] = entities

 72%|██████████████████████▉         | 163340/227733 [00:02<00:00, 71542.86it/s]


IndexError: list index out of range

In [26]:
title_list = list(title_content_lemmatization.keys())
max_value = 4
sememe_network = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            tagme_sememe_dict_v = tagme_sememe_dict[word]
            sememe_freq_max_value = {}
            for sememe in sememes:
                for v in tagme_sememe_dict_v:
                    #print(v)
                    s, l = v.split("#")
                    if s == sememe:
                        sememe_freq_max_value[sememe] = float(l)
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network[word] = sememe_lower
        else:
            sememe_network[word] = sememes
    else:
        continue

In [24]:
max_value = 4
sememe_network = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            sememe_freq_max_value = {}
            for sememe in sememes:
                sememe_freq_max_value[sememe] = sememe_freq[sememe]
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network[word] = sememe_lower
        else:
            sememe_network[word] = sememes
    else:
        continue

In [50]:
sememe_network = {k:v for k,v in sememe_network.items() if len(v)!=0}

In [51]:
print("all lexion size:", len(sememe_network))

all lexion size: 137601


In [ ]:
tfidf_filter

In [28]:
sum([len(v) for v in sememe_network.values()])/len(sememe_network)

1.8371617485000613

In [46]:
sum([len(v) for v in tfidf_filter.values()])/len(tfidf_filter)

2.9999816334027183

In [24]:
sum([len(v) for v in sememe_network_pmi.values()])/len(sememe_network_pmi)

NameError: name 'sememe_network_pmi' is not defined

In [ ]:
163340 1.837106648708216

In [29]:
word_freq = {}
word_set = set()
for words in sememe_network.values():
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

print("vocab_size: ", vocab_size)

vocab_size:  2000


In [72]:
sememe_network["apple.n.0"]

[]

In [73]:
for j,i in enumerate(title_content_lemmatization.keys()):
    if i == "apple.n.0":
        print(j)

87566


In [42]:
title_content_lemmatization["apple.n.1"]

['native',
 'Eurasian',
 'tree',
 'widely',
 'cultivate',
 'in',
 'many',
 'variety',
 'for',
 'it',
 'firm',
 'round',
 'edible',
 'fruit']

In [43]:
sememe_network = {k:set(v) for k, v in sememe_network.items()}

In [48]:
sememe_network_dict = {}
for k, v in sememe_network.items():
    word = k.split(".")[:-2]
    pos = k.split(".")[-2]
    index = k.split(".")[-1]
    if "".join(word) not in sememe_network_dict.keys():
        sememe_network_dict["".join(word)] = []
    sememe_network_dict["".join(word)].append((pos,v))

In [53]:
sememe_network_dict["fruit"]

[('v', {'bear', 'fruit'}),
 ('v', {'bear', 'fruit'}),
 ('n', {'body', 'reproductive', 'seed'})]

In [74]:
np.save("sememe_network_dict_en_wordnet_2000",sememe_network)
np.save("sememe_network_cdv_en_wordnet_2000",vocab)

In [52]:
"fruit" in vocab

True